In [42]:
!pip install catboost

  Using cached https://files.pythonhosted.org/packages/49/98/67ab754fb1579da74c334e4e7683b113149d6f254b73bdce4bdf61a6b52c/catboost-0.8.1.1-cp35-none-manylinux1_x86_64.whl


In [2]:
from sklearn import model_selection

In [3]:
import pandas as pd
import numpy as np

In [4]:
train_df = pd.read_csv("../data/train.csv",index_col = "item_id", parse_dates = ["activation_date"])
test_df = pd.read_csv("../data/test.csv",index_col = "item_id", parse_dates = ["activation_date"])

In [5]:
#Prepare data
N_A_word = "Неопределено"
test_df["param_1"].fillna(N_A_word, inplace=True)
test_df["param_2"].fillna(N_A_word, inplace=True)
test_df["param_3"].fillna(N_A_word, inplace=True)
#TBD заполнить иерархически
test_df["description"].fillna(N_A_word, inplace=True)
train_df["param_1"].fillna(N_A_word, inplace=True)
train_df["param_2"].fillna(N_A_word, inplace=True)
train_df["param_3"].fillna(N_A_word, inplace=True)
train_df["description"].fillna(N_A_word, inplace=True)

In [6]:
train_df["full_prices"] = train_df.groupby(["category_name", "param_1"])["price"].apply(lambda x: x.fillna(x.mean()))
test_df["full_prices"] = test_df.groupby(["category_name", "param_1"])["price"].apply(lambda x: x.fillna(x.mean()))
test_df.drop(["price"], axis=1, inplace=True)
train_df.drop(["price"], axis=1, inplace=True);
train_df["full_prices"].fillna(0, inplace=True)
test_df["full_prices"].fillna(0, inplace=True)

In [7]:
train_df[train_df["full_prices"]==0]
# TODO fill zero prices with bad cost values

,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability,full_prices
item_id,,,,,,,,,,,,,,,,,
97e5dba1a544,ba77f4b48755,Новосибирская область,Новосибирск,Услуги,Предложение услуг,Другое,Неопределено,Неопределено,Обмен места в детском саду,Обменяю место в детском саду N72 на детский са...,4,2017-03-28,Private,NaN,NaN,0.60000,0.0
35f011d65bb8,dfa988975741,Ростовская область,Новочеркасск,Животные,Кошки,Другая,Неопределено,Неопределено,Кошечка ищет дом,Рыжее счастье дом и любящих хозяев .Кошечке 1 ...,15,2017-03-28,Private,88c70b8941bd4230bd9938a038aeb7998d097de7d4d717...,2063.0,0.78503,0.0
54d642ff229a,2f9b389a4cf0,Воронежская область,Воронеж,Услуги,Предложение услуг,Другое,Неопределено,Неопределено,Подключение uber (Воронеж),Предлагаем работу водителем (сменный график)./...,10,2017-03-21,Company,d3f60b28ba7072f9722c98d668918df864f5ee160eb786...,2266.0,0.00000,0.0
db6143e5d1b4,8467734ce392,Краснодарский край,Сочи,Хобби и отдых,Книги и журналы,Учебная литература,Неопределено,Неопределено,Учебники 8 класса,Неопределено,58,2017-03-20,Private,ecf8e9498c71bb84206936ef1cceade666818e024a9e99...,1050.0,0.18907,0.0
ee1d51eb4e6c,c3e00b4a0f99,Иркутская область,Иркутск,Животные,Кошки,Другая,Неопределено,Неопределено,Отдам милых котят,"Отдам очень милых котят. Им 1,5 месяца приучен...",35,2017-03-25,Private,2cba8460919c67557be36e9add6fee3002f1afe61e08fd...,2054.0,0.39355,0.0
c5bca1a63c11,c5d59c758a04,Башкортостан,Кумертау,Животные,Кошки,Британская,Неопределено,Неопределено,Отдадим кота в хорошие руки,Отдадим котика в хорошие руки в связи с семейн...,5,2017-03-16,Private,e138e00cea4979c7894036c16f6708bcda1d54a5b580c0...,2054.0,0.78503,0.0
1c10b9025e48,add013223f9b,Самарская область,Самара,Услуги,Предложение услуг,Уборка,Вывоз мусора,Неопределено,"Вывезу мусор камаз,ЗИЛ,газель","УСЛУГИ по ВЫВОЗу МУСОРа, УСЛУГИ ГРУЗЧИКОВ, ПРИ...",2,2017-03-18,Private,4d5bcf916ebdd50eed2c51e5414674c8578df7420a02b1...,1207.0,0.40000,0.0
a3d1d1ba909e,ecf3f232be6f,Татарстан,Казань,Для дома и дачи,Растения,Неопределено,Неопределено,Неопределено,Бесплатно отдаю растения,"На площадке по адресу г. Казань, Ул. Четаева, ...",61,2017-03-15,Private,a4d46e4166a6699dd5af1bb3140b1b07cbde237c6a08ec...,1733.0,0.38925,0.0
b65c8ea48f4e,3025f6c20e3d,Волгоградская область,Волгоград,Для дома и дачи,Мебель и интерьер,Подставки и тумбы,Неопределено,Неопределено,Тумба бесплатно,Отдам бесплатно.В реальности смотрится лучше.,236,2017-03-22,Private,19fcbf94ae776a5fe49b74aeb18cff9a221af56d3fad7b...,1456.0,0.86521,0.0


In [8]:
def create_time_vars(df):
    df["Weekday"] = df['activation_date'].dt.weekday
    df["Weekd of Year"] = df['activation_date'].dt.week
    df["Day of Month"] = df['activation_date'].dt.day
create_time_vars(train_df)
create_time_vars(test_df)
train_df.drop(["activation_date"],axis=1,inplace=True)
test_df.drop(["activation_date"],axis=1,inplace=True)

# Prepare data

In [9]:
train_df['norm_price'] = pd.Series(0,index=train_df.index)
test_df['norm_price'] = pd.Series(0,index=test_df.index)

In [10]:
y = train_df["deal_probability"]
train_df.drop(["deal_probability"], axis=1, inplace=True)

In [11]:
from sklearn.preprocessing import StandardScaler
categories = train_df["category_name"].unique()
for cat in categories:
    scaler = StandardScaler()
    norm_price = scaler.fit_transform(train_df[train_df["category_name"]==cat]["full_prices"].values.reshape(-1,1))
    train_df.loc[train_df["category_name"]==cat,"norm_price"] = norm_price
    norm_price_test = scaler.transform(test_df[test_df["category_name"]==cat]["full_prices"].values.reshape(-1,1))
    test_df.loc[test_df["category_name"]==cat,"norm_price"] = norm_price_test

In [12]:
train_df.drop(["full_prices"], axis = 1, inplace=True)
test_df.drop(["full_prices"], axis = 1, inplace=True)

In [13]:
train_has_image = train_df["image"].notnull().astype('int')
test_has_image = test_df["image"].notnull().astype('int')

In [14]:
from sklearn.preprocessing import LabelBinarizer
usertLab = LabelBinarizer()
train_usert_oh = usertLab.fit_transform(train_df["user_type"])
test_usert_oh = usertLab.transform(test_df["user_type"])

In [15]:
def get_train_test_oh(column, sparse=True):
    labeler = LabelBinarizer(sparse_output=sparse)
    train_lbl_oh = labeler.fit_transform(train_df[column])
    test_lbl_oh = labeler.transform(test_df[column])
    return train_lbl_oh, test_lbl_oh

train_pr1_oh, test_pr1_oh = get_train_test_oh("param_1")

In [16]:
train_cat_oh, test_cat_oh = get_train_test_oh("category_name")

In [17]:
train_pcat_oh, test_pcat_oh = get_train_test_oh("parent_category_name")

In [18]:
train_city_oh, test_city_oh = get_train_test_oh("city")

In [19]:
train_region_oh, test_region_oh = get_train_test_oh("region")

In [20]:
train_wd_oh, test_wd_oh = get_train_test_oh("Weekday")

In [21]:
train_week_oh, test_week_oh = get_train_test_oh("Weekd of Year")

In [22]:
train_day_oh, test_day_oh = get_train_test_oh("Day of Month")

In [23]:
def free_in(v):
    if "бесплатн" in v:
        return 1
    else: 
        return 0
train_free_title = train_df["title"].apply(free_in).values.reshape(-1,1)
train_free_desc = train_df["description"].apply(free_in).values.reshape(-1,1)
test_free_title = test_df["title"].apply(free_in).values.reshape(-1,1)
test_free_desc = test_df["description"].apply(free_in).values.reshape(-1,1)

In [24]:
train_df["image_top_1"].fillna(0, inplace=True)
test_df["image_top_1"].fillna(0, inplace=True)

In [25]:
train_img_cls, test_img_cls = get_train_test_oh("image_top_1")

In [26]:
train_pr2_oh, test_pr2_oh = get_train_test_oh("param_2")

In [27]:
train_pr3_oh, test_pr3_oh = get_train_test_oh("param_3")

In [28]:
train_pr3_oh

<1503424x1220 sparse matrix of type '<class 'numpy.int64'>'
	with 1503424 stored elements in Compressed Sparse Row format>

# Text Features

In [29]:
train_titles = train_df["title"]
test_titles = test_df["title"]
train_desc = train_df["description"]
test_desc = test_df["description"]

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
train_vct_title = TfidfVectorizer()
train_title_tfidf = train_vct_title.fit_transform(train_titles)
test_title_tfidf = train_vct_title.transform(test_titles)

In [31]:
train_vct_desc = TfidfVectorizer()
train_desc_tfidf = train_vct_desc.fit_transform(train_desc)
test_desc_tfidf = train_vct_desc.transform(test_desc)

In [32]:
from scipy.sparse import hstack
train_text_x = hstack([train_title_tfidf, train_desc_tfidf])
test_text_x = hstack([test_title_tfidf, test_desc_tfidf])

In [33]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
rcl_text = Ridge(random_state=17)
scores = cross_val_score(rcl_text, train_text_x, y, n_jobs = -1, verbose=1,  scoring="neg_mean_squared_error")

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.6min finished


In [34]:
np.mean(np.sqrt(-scores))

0.23578176616430954

# Ridge regression

In [35]:
from scipy.sparse import hstack
train_x = hstack([train_pr3_oh,train_pr2_oh,train_img_cls,train_free_title, train_free_desc,train_df["norm_price"].values.reshape(-1,1),train_cat_oh, train_city_oh, train_has_image.values.reshape(-1,1), train_pcat_oh, train_pr1_oh, train_region_oh, train_usert_oh])
test_x = hstack([test_pr3_oh,test_pr2_oh,test_img_cls,test_free_title, test_free_desc,test_df["norm_price"].values.reshape(-1,1),test_cat_oh, test_city_oh, test_has_image.values.reshape(-1,1), test_pcat_oh, test_pr1_oh, test_region_oh, test_usert_oh])

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_x, y, test_size=0.2, random_state=42)

In [37]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
import numpy as np
rcl = Ridge(random_state=101)
scores = cross_val_score(rcl, X_train, y_train, n_jobs = -1, verbose=1,  scoring="neg_mean_squared_error")
np.mean(np.sqrt(-scores))

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  4.7min finished


0.2332616851382691

# Catboost training

In [38]:
train_df.head(1)

,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,item_seq_number,user_type,image,image_top_1,Weekday,Weekd of Year,Day of Month,norm_price
item_id,,,,,,,,,,,,,,,,,,
b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,Неопределено,Неопределено,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",2,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,1,13,28,-0.027701


In [39]:
cat_features_train = train_df.drop(["user_id","title","description","item_seq_number","image"], axis=1)
cat_features_test = test_df.drop(["user_id","title","description","item_seq_number","image"], axis=1)

In [40]:
cat_features_train.columns

Index(['region', 'city', 'parent_category_name', 'category_name', 'param_1',
       'param_2', 'param_3', 'user_type', 'image_top_1', 'Weekday',
       'Weekd of Year', 'Day of Month', 'norm_price'],
      dtype='object')

In [43]:
from catboost import Pool, CatBoostRegressor

train_pool = Pool(cat_features_train, y, cat_features=list(range(len(cat_features_train.columns)-1)))
test_pool = Pool(cat_features_test, cat_features=list(range(len(cat_features_test.columns)-1))) 

# specify the training parameters 
model = CatBoostRegressor(iterations=50, depth=3, learning_rate=0.5, loss_function='RMSE')

In [44]:
from catboost import cv
parameters = {'iterations': 50, 'depth':3, 'learning_rate':0.5, 'loss_function':'RMSE'}
cb_scores = cv(params=parameters, pool=train_pool)
np.sqrt(cb_scores)

0:	learn: 0.2574845	test: 0.2574794	best: 0.2574794 (0)	total: 2.88s	remaining: 2m 20s
1:	learn: 0.2448537	test: 0.2448268	best: 0.2448268 (1)	total: 5.2s	remaining: 2m 4s
2:	learn: 0.2407261	test: 0.2406681	best: 0.2406681 (2)	total: 7.81s	remaining: 2m 2s
3:	learn: 0.2387033	test: 0.2385889	best: 0.2385889 (3)	total: 10.2s	remaining: 1m 57s
4:	learn: 0.2375511	test: 0.2373822	best: 0.2373822 (4)	total: 12.7s	remaining: 1m 54s
5:	learn: 0.2367331	test: 0.2365039	best: 0.2365039 (5)	total: 15.1s	remaining: 1m 51s
6:	learn: 0.2363319	test: 0.2360911	best: 0.2360911 (6)	total: 17.5s	remaining: 1m 47s
7:	learn: 0.2359199	test: 0.2356865	best: 0.2356865 (7)	total: 19.8s	remaining: 1m 43s
8:	learn: 0.2356487	test: 0.2354047	best: 0.2354047 (8)	total: 22.3s	remaining: 1m 41s
9:	learn: 0.2354245	test: 0.2351520	best: 0.2351520 (9)	total: 24.5s	remaining: 1m 37s
10:	learn: 0.2348839	test: 0.2345835	best: 0.2345835 (10)	total: 26.7s	remaining: 1m 34s
11:	learn: 0.2345828	test: 0.2342683	best: 0

,test-RMSE-mean,test-RMSE-std,train-RMSE-mean,train-RMSE-std
0,0.507424,0.022899,0.507429,0.019769
1,0.494800,0.026264,0.494827,0.015343
2,0.490579,0.021390,0.490638,0.019843
3,0.488456,0.025005,0.488573,0.015867
4,0.487219,0.025648,0.487392,0.014680
5,0.486317,0.015447,0.486552,0.024899
6,0.485892,0.016347,0.486140,0.023961
7,0.485476,0.017381,0.485716,0.023221
8,0.485185,0.019930,0.485437,0.021365
9,0.484925,0.021856,0.485206,0.019843


In [45]:
model.fit(train_pool)
cb_preds = model.predict(test_pool)

0:	learn: 0.2573591	total: 1.05s	remaining: 51.5s
1:	learn: 0.2447182	total: 1.69s	remaining: 40.6s
2:	learn: 0.2407734	total: 2.29s	remaining: 35.8s
3:	learn: 0.2386657	total: 3.1s	remaining: 35.6s
4:	learn: 0.2376566	total: 3.93s	remaining: 35.4s
5:	learn: 0.2370942	total: 4.84s	remaining: 35.5s
6:	learn: 0.2365874	total: 5.78s	remaining: 35.5s
7:	learn: 0.2362748	total: 6.77s	remaining: 35.5s
8:	learn: 0.2358183	total: 7.66s	remaining: 34.9s
9:	learn: 0.2355186	total: 8.57s	remaining: 34.3s
10:	learn: 0.2353178	total: 9.44s	remaining: 33.5s
11:	learn: 0.2351896	total: 10.3s	remaining: 32.7s
12:	learn: 0.2347782	total: 11.2s	remaining: 32s
13:	learn: 0.2346608	total: 12.1s	remaining: 31.1s
14:	learn: 0.2342113	total: 13s	remaining: 30.4s
15:	learn: 0.2340982	total: 14s	remaining: 29.8s
16:	learn: 0.2339360	total: 14.9s	remaining: 28.9s
17:	learn: 0.2338814	total: 15.8s	remaining: 28s
18:	learn: 0.2337482	total: 16.8s	remaining: 27.4s
19:	learn: 0.2334922	total: 17.8s	remaining: 26.6s

# prepare stacking values

In [46]:
#ridge
from sklearn.model_selection import cross_val_predict
ridge_scores = cross_val_predict(rcl, train_x, y, n_jobs = -1, verbose=1)

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  7.0min finished


In [47]:
#text
text_scores = cross_val_predict(rcl, train_text_x, y, n_jobs = -1, verbose=1)

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.6min finished


In [60]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)
cb_preds = []
for train_index, test_index in kf.split(cat_features_train.values):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train_cv, X_test_cv = cat_features_train.values[train_index], cat_features_train.values[test_index]
    y_train_cv, y_test_cv = y[train_index], y[test_index]
    cv_train_pool = Pool(X_train_cv, y_train_cv, cat_features=list(range(len(cat_features_train.columns)-1)))
    cv_test_pool = Pool(X_test_cv, cat_features=list(range(len(cat_features_test.columns)-1))) 
    cv_model = CatBoostRegressor(iterations=50, depth=3, learning_rate=0.5, loss_function='RMSE')
    cv_model.fit(cv_train_pool)
    cb_preds.append(cv_model.predict(cv_test_pool))

TRAIN: [ 501142  501143  501144 ... 1503421 1503422 1503423] TEST: [     0      1      2 ... 501139 501140 501141]
0:	learn: 0.2575395	total: 582ms	remaining: 28.5s
1:	learn: 0.2449077	total: 1.12s	remaining: 27s
2:	learn: 0.2409724	total: 1.52s	remaining: 23.8s
3:	learn: 0.2388517	total: 1.99s	remaining: 22.9s
4:	learn: 0.2378248	total: 2.27s	remaining: 20.4s
5:	learn: 0.2371776	total: 2.75s	remaining: 20.2s
6:	learn: 0.2367127	total: 3.22s	remaining: 19.8s
7:	learn: 0.2360311	total: 3.75s	remaining: 19.7s
8:	learn: 0.2356690	total: 4.26s	remaining: 19.4s
9:	learn: 0.2354611	total: 4.71s	remaining: 18.9s
10:	learn: 0.2352217	total: 5.28s	remaining: 18.7s
11:	learn: 0.2348948	total: 5.76s	remaining: 18.2s
12:	learn: 0.2347525	total: 6.21s	remaining: 17.7s
13:	learn: 0.2346204	total: 6.64s	remaining: 17.1s
14:	learn: 0.2343359	total: 7.17s	remaining: 16.7s
15:	learn: 0.2341477	total: 7.7s	remaining: 16.4s
16:	learn: 0.2340458	total: 8.24s	remaining: 16s
17:	learn: 0.2338790	total: 8.65s

In [64]:
cb_preds = np.concatenate(cb_preds)
cb_preds.shape

(1503424,)

In [68]:
combined_scores = np.concatenate([ridge_scores.reshape(-1,1),text_scores.reshape(-1,1),cb_preds.reshape(-1,1)], axis=1)
combined_scores.shape

(1503424, 3)

In [69]:
from xgboost import XGBRegressor
xgrgr = XGBRegressor()
xgrgr.fit(combined_scores, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [71]:
final_ridge = Ridge(random_state=101)
final_ridge.fit(train_x, y)
y_pred = final_ridge.predict(test_x)

In [72]:
final_ridge_text = Ridge(random_state=17)
final_ridge_text.fit(train_text_x, y)
y_pred_text = final_ridge_text.predict(test_text_x)

In [73]:
cb_final_model = CatBoostRegressor(iterations=100, depth=3, learning_rate=0.5, loss_function='RMSE')
cb_final_model.fit(train_pool)
y_pred_cb = cb_final_model.predict(test_pool)

0:	learn: 0.2573489	total: 872ms	remaining: 42.7s
1:	learn: 0.2447328	total: 1.65s	remaining: 39.6s
2:	learn: 0.2408190	total: 2.34s	remaining: 36.7s
3:	learn: 0.2387125	total: 3.06s	remaining: 35.2s
4:	learn: 0.2377200	total: 3.6s	remaining: 32.4s
5:	learn: 0.2371567	total: 4.29s	remaining: 31.5s
6:	learn: 0.2365736	total: 5.08s	remaining: 31.2s
7:	learn: 0.2362644	total: 5.81s	remaining: 30.5s
8:	learn: 0.2357144	total: 6.64s	remaining: 30.2s
9:	learn: 0.2354055	total: 7.29s	remaining: 29.2s
10:	learn: 0.2352367	total: 7.94s	remaining: 28.1s
11:	learn: 0.2350497	total: 8.52s	remaining: 27s
12:	learn: 0.2348502	total: 9.24s	remaining: 26.3s
13:	learn: 0.2346326	total: 10.1s	remaining: 25.9s
14:	learn: 0.2344583	total: 10.8s	remaining: 25.2s
15:	learn: 0.2342097	total: 11.6s	remaining: 24.7s
16:	learn: 0.2339762	total: 12.4s	remaining: 24.1s
17:	learn: 0.2338933	total: 13.1s	remaining: 23.2s
18:	learn: 0.2338371	total: 13.8s	remaining: 22.5s
19:	learn: 0.2337395	total: 14.7s	remaining:

In [75]:
y_test_final = np.concatenate([y_pred.reshape(-1,1),y_pred_text.reshape(-1,1), y_pred_cb.reshape(-1,1)],axis=1)

In [76]:
lgb_score = xgrgr.predict(y_test_final)

In [78]:
test_item_id = test_df.index
sub_df = pd.DataFrame({"item_id":test_item_id})
sub_df["deal_probability"] = lgb_score.clip(0,1)
sub_df.to_csv("/tmp/baseline_avito_xgb_cb.csv", index=False)

In [81]:
test_item_id = test_df.index
sub_df = pd.DataFrame({"item_id":test_item_id})
sub_df["deal_probability"] = y_pred_cb.clip(0,1)
sub_df.to_csv("/tmp/baseline_avito_xgb_cb.csv", index=False)